In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df=pd.read_csv('../input/titanic/train.csv')
test_df=pd.read_csv('../input/titanic/test.csv')
test1_df=test_df.copy()

In [ ]:
train_df.columns

In [ ]:
train_df.shape

In [ ]:
train_df['Embarked'].unique()

In [ ]:
train_df.drop(['PassengerId','Name','Pclass','Ticket','Fare','Cabin'],axis=1,inplace=True)

In [ ]:
train_df

In [ ]:
train_df.isnull().sum()

In [ ]:
(train_df.isnull().sum()/train_df.shape[0])*100

In [ ]:
train_df['Age'].describe()

In [ ]:
train_df['Age'].fillna(value=train_df['Age'].mean(),inplace=True)

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.dropna(axis=0,inplace=True)

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['Sex'].value_counts()

In [ ]:
train_df['Survived'].value_counts()

In [ ]:
sns.countplot(train_df['Sex'],hue=train_df['Survived'])

In [ ]:
plt.hist(train_df['Age'],bins=100)
plt.title('AGE',)
plt.show()


### Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
train_df['Sex']=le.fit_transform(train_df['Sex'])

In [ ]:
train_df

In [ ]:
train_df['Embarked']=le.fit_transform(train_df['Embarked'])

In [ ]:
train_df

### Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

In [ ]:
featurescale=['Age']
train_df[featurescale]=ss.fit_transform(train_df[featurescale])

In [ ]:
train_df

In [ ]:
x=train_df.drop(['Survived'],axis=1)
y=train_df['Survived']

### Feature Importance

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
select=ExtraTreesRegressor()
select.fit(x,y)

In [ ]:
print(select.feature_importances_)

In [ ]:
rank_features=pd.Series(select.feature_importances_,index=x.columns)

In [ ]:
rank_features

In [ ]:
rank_features.plot(kind='barh')

## Correlation

In [ ]:
train_df.corr()

In [ ]:
import seaborn as sns
corr=train_df.iloc[:,:-1].corr()
top_features=corr.index
plt.figure(figsize=(20,20))
sns.heatmap(train_df[top_features].corr(),annot=True)

By seeing the vlaues we can say that there is no correlation among the features

In [ ]:
sns.pairplot(train_df)

### Model Building

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=37)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train,y_train)

In [ ]:
y_pred=rf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

# Hyperparameter tuning

### RandomizedSearch CV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[200,400,600,800,1000,1200]

max_features=['auto', 'sqrt','log2']

max_depth = [int(x) for x in np.linspace(10, 1000,10)]

min_samples_split = [2, 5, 10,14]

min_samples_leaf = [1, 2, 4,6,8]

random_grid={'n_estimators':n_estimators,
            'max_features':max_features,
            'max_depth':max_depth,
            'min_samples_split':min_samples_split,
            'min_samples_leaf':min_samples_leaf,
            'criterion':['entropy','gini']}


In [ ]:
print(random_grid)

In [ ]:
rf1=RandomForestClassifier()
rf_randomized=RandomizedSearchCV(estimator=rf1,param_distributions=random_grid,n_iter=500,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)

In [ ]:
rf_randomized.fit(x_train,y_train)

In [ ]:
rf_randomized.best_params_

In [ ]:
rf_randomized.best_estimator_

### Best Model

In [ ]:
best_model=rf_randomized.best_estimator_

In [ ]:
y_pred1=best_model.predict(x_test)
accuracy_score(y_test,y_pred1)

# Test Data

In [ ]:
test_df.drop(['PassengerId','Name','Pclass','Ticket','Fare','Cabin'],axis=1,inplace=True)

In [ ]:
test_df['Age'].fillna(value=test_df['Age'].mean(),inplace=True)

In [ ]:
test_df.isnull().sum()

In [ ]:
featurescale=['Age']
test_df[featurescale]=ss.fit_transform(test_df[featurescale])

In [ ]:
test_df['Sex']=le.fit_transform(test_df['Sex'])

In [ ]:
test_df['Embarked']=le.fit_transform(test_df['Embarked'])

In [ ]:
predictions=best_model.predict(test_df)

In [ ]:
output = pd.DataFrame({'PassengerId': test1_df.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")